<a href="https://colab.research.google.com/github/Donutson/hackathon-abidjan-data-team-1/blob/main/Conseil_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BeOpenIT Hackathon-Data-IA-ML Team Machine Learnia
Construction des modèles pour la mise en production des 2 derniers buts

## Data loading and cleaning

### Loading

In [1]:
# import
import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
print("Import done")

Import done


In [2]:
# loading 
climate = pd.read_csv("climate.csv", parse_dates=["DATE"])
airport = pd.read_csv("AIRPORT_ONTIME_REPORTING.csv", parse_dates=["FL_DATE"])
airport_climate = pd.merge(airport, climate, right_on="DATE", left_on="FL_DATE")

# only chicago
dataset = airport_climate[airport_climate['ORIGIN_CITY_NAME'].str.contains("Chicago") | airport_climate['DEST_CITY_NAME'].str.contains("Chicago") ].copy()
print("dataset build")

dataset build


In [3]:
#Suppression des variables derivées et non contextuelles après analyse (correlation, logique,....)
dataset.drop(['ORIGIN_STATE_NM', 'ORIGIN_WAC','DEST', 'DEST_STATE_NM', 'DEST_WAC', 'NAS_DELAY','SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'DIVERTED', 'YEAR', 'TAXI_OUT', 'ARR_DELAY_GROUP','ARR_TIME_BLK',  'DISTANCE_GROUP', 'CARRIER_DELAY',], axis='columns', inplace=True)
dataset.drop(['TAVG', 'ORIGIN_CITY_NAME','DEST_CITY_NAME', 'DEP_DELAY_GROUP', 'DEP_TIME_BLK',  'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME','TMIN','STATION','NAME','LATITUDE','LONGITUDE','ELEVATION', 'DATE', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'FL_DATE', 'TAIL_NUM', 'ORIGIN_AIRPORT_ID',  'DEST_AIRPORT_ID', 'ORIGIN'], axis='columns', inplace=True)
dataset.drop(['ARR_DELAY', 'DEP_DELAY'], axis='columns', inplace=True)
dataset.drop(['OP_CARRIER_FL_NUM','TAXI_IN','Unnamed: 49','ORIGIN_STATE_ABR','DEST_STATE_ABR','DEST_AIRPORT_SEQ_ID','DEST_CITY_MARKET_ID'], axis='columns', inplace=True)
print("unuseful features has been drop")

unuseful features has been drop


### Cleanning

Imputation with median

In [4]:
# Remplacer les NaN en utilisant la médiane
median = dataset['AWND'].median()
dataset['AWND'].fillna(median, inplace=True)

median = dataset['PRCP'].median()
dataset['PRCP'].fillna(median, inplace=True)

median = dataset['SNOW'].median()
dataset['SNOW'].fillna(median, inplace=True)

median = dataset['TMAX'].median()
dataset['TMAX'].fillna(median, inplace=True)

median = dataset['DEP_TIME'].median()
dataset['DEP_TIME'].fillna(median, inplace=True)

median = dataset['CRS_DEP_TIME'].median()
dataset['CRS_DEP_TIME'].fillna(median, inplace=True)

median = dataset['DEP_DEL15'].median()
dataset['DEP_DEL15'].fillna(median, inplace=True)

median = dataset['ARR_TIME'].median()
dataset['ARR_TIME'].fillna(median, inplace=True)

median = dataset['CRS_ARR_TIME'].median()
dataset['CRS_ARR_TIME'].fillna(median, inplace=True)

median = dataset['ARR_DEL15'].median()
dataset['ARR_DEL15'].fillna(median, inplace=True)

median = dataset['CANCELLED'].median()
dataset['CANCELLED'].fillna(median, inplace=True)

median = dataset['AIR_TIME'].median()
dataset['AIR_TIME'].fillna(median, inplace=True)

median = dataset['FLIGHTS'].median()
dataset['FLIGHTS'].fillna(median, inplace=True)

median = dataset['DISTANCE'].median()
dataset['DISTANCE'].fillna(median, inplace=True)

median = dataset['WEATHER_DELAY'].median()
dataset['WEATHER_DELAY'].fillna(median, inplace=True)

print("dataset has been cleaned")

dataset has been cleaned


## Modeling

In [5]:
# model selection
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split

# preprocessing
from sklearn.preprocessing import StandardScaler

# regressors
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

print('All are imported')

All are imported


### But 3: Predire le temps d'arrivée reelle



In [18]:
# target features
y=dataset['ARR_TIME']
X = dataset.loc[:, dataset.columns != 'ARR_TIME']
X = X.loc[:, X.columns != 'CANCELLED']
X = X.loc[:, X.columns != 'ARR_DEL15']
X = X.loc[:, X.columns != 'CRS_ARR_TIME']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

list(X)

['DAY_OF_WEEK',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'DEP_DEL15',
 'AIR_TIME',
 'FLIGHTS',
 'DISTANCE',
 'WEATHER_DELAY',
 'AWND',
 'PRCP',
 'SNOW',
 'TMAX']

### Linear regression

In [7]:
model1 = LinearRegression()
print("cv r2 score:",np.mean(cross_val_score(model1, X, y, cv=10, scoring='r2')))

cv r2 score: 0.3880578546370755


### Decision tree

In [8]:
params = {"max_depth": range(2,15)}
model2 = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=params, scoring="r2", cv=10)
model2.fit(X, y)
print("cv r2 score:",model2.best_score_)

cv r2 score: 0.7736768388234233


### KNN

In [9]:
params = {"n_neighbors": range(1,10), "p": range(1,3)}
model3 = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=params, scoring="r2", cv=10)
model3.fit(X, y)
print("cv r2 score:",model2.best_score_)

cv r2 score: 0.7736768388234233


### RF

In [10]:
params = {"n_estimators": range(1,10), "max_depth": range(1,5)}
model4 = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, scoring="r2", cv=10)
model4.fit(X, y)
print("cv r2 score:",model2.best_score_)

cv r2 score: 0.7736768388234233


In [11]:
# best model save
from joblib import dump
dump(model2, "goal3.model")

['goal3.model']

### But 4 : Predire la date réelle de depart 

In [25]:
# target and featurees
y=dataset['DEP_TIME']
X = dataset.loc[:, dataset.columns != 'ARR_DEL15']
X = X.loc[:, X.columns != 'CANCELLED']
X = X.loc[:, X.columns != 'DEP_TIME']
X = X.loc[:, X.columns != 'CRS_DEP_TIME']
X = X.loc[:, X.columns != 'DEP_DEL15']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

list(X)

['DAY_OF_WEEK',
 'CRS_ARR_TIME',
 'ARR_TIME',
 'AIR_TIME',
 'FLIGHTS',
 'DISTANCE',
 'WEATHER_DELAY',
 'AWND',
 'PRCP',
 'SNOW',
 'TMAX']

### Linear regression

In [13]:
model1 = LinearRegression()
print("cv r2 score:",np.mean(cross_val_score(model1, X, y, cv=10, scoring='r2')))

cv r2 score: 0.6001875676913626


### Decision tree

In [14]:
params = {"max_depth": range(2,15)}
model2 = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=params, scoring="r2", cv=10)
model2.fit(X, y)
print("cv r2 score:",model2.best_score_)

cv r2 score: 0.9210604684526468


### KNN

In [15]:
params = {"n_neighbors": range(1,10), "p": range(1,3)}
model3 = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=params, scoring="r2", cv=10)
model3.fit(X, y)
print("cv r2 score:",model2.best_score_)

cv r2 score: 0.9210604684526468


### RF

In [16]:
params = {"n_estimators": range(1,10), "max_depth": range(1,5)}
model4 = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, scoring="r2", cv=10)
model4.fit(X, y)
print("cv r2 score:",model2.best_score_)

cv r2 score: 0.9210604684526468


In [17]:
dump(model2, "goal4.model")

['goal4.model']